get NPP from MYD MOD17AHGF

year: 2001 - 2020

Point: Japanese 4rd time Mesh 0.5 km centroid

## NNP Extraction
Net Primary Productivity

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from osgeo import gdal
import os
import glob
import rasterio

In [6]:
#year = 2015
aimFolder = "F:/17_Article/01_Data/02_NetPrimaryProductivity"
os.mkdir(aimFolder + "\\temp")

In [9]:
def mergeAndResampleNppRaster(year, aimFolder):
    #"""
    #Merge MOD and MYD,
    #Resample the raster with 0.004 degrees
    #"""
    rasterFile = glob.glob("F:/17_Article/01_Data/02_NetPrimaryProductivity/Net_PP_Yearly_500m_v6/Npp/*"+str(year) +"_001.tif")

    if year > 2001:
        tifflayer_0 = gdal.Open(rasterFile[0], gdal.GA_ReadOnly)
        tifflayer_1 = gdal.Open(rasterFile[1], gdal.GA_ReadOnly)

        nband = 1
        geotransform = tifflayer_0.GetGeoTransform()
        spatialreference = tifflayer_0.GetProjection()

        tifflayer_0_array = tifflayer_0.ReadAsArray()
        tifflayer_0_array = np.array(tifflayer_0_array)
        tifflayer_0_array = tifflayer_0_array.astype(float)
        tifflayer_0_array[tifflayer_0_array == 65535] = np.nan
        tifflayer_0_array[tifflayer_0_array > 32760] = 0

        tifflayer_1_array = tifflayer_1.ReadAsArray()
        tifflayer_1_array = np.array(tifflayer_1_array)
        tifflayer_1_array = tifflayer_1_array.astype(float)
        tifflayer_1_array[tifflayer_1_array == 65535] = np.nan
        tifflayer_1_array[tifflayer_1_array > 32760] = 0

        tifflayer_double_array = np.array([tifflayer_0_array, tifflayer_1_array])
        tifflayer_mean_array = np.nanmean(tifflayer_double_array, axis = 0)
    else:
        tifflayer_0 = gdal.Open(rasterFile[0], gdal.GA_ReadOnly)

        nband = 1
        geotransform = tifflayer_0.GetGeoTransform()
        spatialreference = tifflayer_0.GetProjection()

        tifflayer_0_array = tifflayer_0.ReadAsArray()
        tifflayer_0_array = np.array(tifflayer_0_array)
        tifflayer_0_array = tifflayer_0_array.astype(float)
        tifflayer_0_array[tifflayer_0_array == 65535] = np.nan
        tifflayer_0_array[tifflayer_0_array > 32760] = 0
        tifflayer_mean_array = tifflayer_0_array 


    ncol = tifflayer_mean_array.shape[1]
    nrow = tifflayer_mean_array.shape[0]

    temp_file = aimFolder + "/temp/temp_file_" + str(year) + ".tif"

    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(temp_file, ncol, nrow, nband, gdal.GDT_Float32)
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(tifflayer_mean_array)
    dst_dataset = None

    original_temp_raster = gdal.Open(temp_file, gdal.GA_ReadOnly)
    raster_rprj = gdal.Warp(aimFolder + "/temp/temp_file_" + str(year) + "0008.tif", original_temp_raster, dstSRS = "EPSG:4326", 
                            xRes = 0.004, yRes = 0.004, resampleAlg = "average")
    original_temp_raster = None
    raster_rprj = None


In [7]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

Parallel(n_jobs=10)(delayed(mergeAndResampleNppRaster)(int(year), aimFolder) for year in np.linspace(2002, 2020, 20))

IndexError: list index out of range

In [8]:
np.linspace(2001, 2020, 20)

array([2001., 2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009.,
       2010., 2011., 2012., 2013., 2014., 2015., 2016., 2017., 2018.,
       2019., 2020.])

In [15]:
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")

In [73]:
#os.remove(temp_file)
#os.remove(aimFolder + "/temp/temp_file_0008.tif")

#os.rmdir(aimFolder + "\\temp")

In [75]:
def nppExtraction(year, aimFolder, coords_extration):
    # this is to extract npp of each eyar of each mesh point
    rasterFile = rasterio.open(aimFolder + "/temp/temp_file_" + str(year) + "0004.tif")
    rasterArray = rasterFile.read(1)

    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
        
    valueArray = np.array(valueArray)
    coords_extration['NPP_'+str(year)] = valueArray

    rasterFile = None
    return coords_extration

In [76]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

aimFolder = "F:/17_Article/01_Data/02_NetPrimaryProductivity"
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")
nppResult = Parallel(n_jobs=4)(delayed(nppExtraction)(int(year), aimFolder, coords_extration) for year in np.linspace(2001, 2020, 20))

In [77]:
nppResult_2001 = nppResult[0]
nppResult_2002 = nppResult[1]
nppResult_2003 = nppResult[2]
nppResult_2004 = nppResult[3]
nppResult_2005 = nppResult[4]
nppResult_2006 = nppResult[5]
nppResult_2007 = nppResult[6]
nppResult_2008 = nppResult[7]
nppResult_2009 = nppResult[8]
nppResult_2010 = nppResult[9]
nppResult_2011 = nppResult[10]
nppResult_2012 = nppResult[11]
nppResult_2013 = nppResult[12]
nppResult_2014 = nppResult[13]
nppResult_2015 = nppResult[14]
nppResult_2016 = nppResult[15]
nppResult_2017 = nppResult[16]
nppResult_2018 = nppResult[17]
nppResult_2019 = nppResult[18]
nppResult_2020 = nppResult[19]

In [78]:
nppResult_2001['year'] = 2001
nppResult_2002['year'] = 2002
nppResult_2003['year'] = 2003
nppResult_2004['year'] = 2004
nppResult_2005['year'] = 2005
nppResult_2006['year'] = 2006
nppResult_2007['year'] = 2007
nppResult_2008['year'] = 2008
nppResult_2009['year'] = 2009
nppResult_2010['year'] = 2010
nppResult_2011['year'] = 2011
nppResult_2012['year'] = 2012
nppResult_2013['year'] = 2013
nppResult_2014['year'] = 2014
nppResult_2015['year'] = 2015
nppResult_2016['year'] = 2016
nppResult_2017['year'] = 2017
nppResult_2018['year'] = 2018
nppResult_2019['year'] = 2019
nppResult_2020['year'] = 2020

In [79]:
nppResult_2001 = pd.DataFrame(nppResult_2001.drop(columns='geometry'))
nppResult_2002 = pd.DataFrame(nppResult_2002.drop(columns='geometry'))
nppResult_2003 = pd.DataFrame(nppResult_2003.drop(columns='geometry'))
nppResult_2004 = pd.DataFrame(nppResult_2004.drop(columns='geometry'))
nppResult_2005 = pd.DataFrame(nppResult_2005.drop(columns='geometry'))
nppResult_2006 = pd.DataFrame(nppResult_2006.drop(columns='geometry'))
nppResult_2007 = pd.DataFrame(nppResult_2007.drop(columns='geometry'))
nppResult_2008 = pd.DataFrame(nppResult_2008.drop(columns='geometry'))
nppResult_2009 = pd.DataFrame(nppResult_2009.drop(columns='geometry'))
nppResult_2010 = pd.DataFrame(nppResult_2010.drop(columns='geometry'))
nppResult_2011 = pd.DataFrame(nppResult_2011.drop(columns='geometry'))
nppResult_2012 = pd.DataFrame(nppResult_2012.drop(columns='geometry'))
nppResult_2013 = pd.DataFrame(nppResult_2013.drop(columns='geometry'))
nppResult_2014 = pd.DataFrame(nppResult_2014.drop(columns='geometry'))
nppResult_2015 = pd.DataFrame(nppResult_2015.drop(columns='geometry'))
nppResult_2016 = pd.DataFrame(nppResult_2016.drop(columns='geometry'))
nppResult_2017 = pd.DataFrame(nppResult_2017.drop(columns='geometry'))
nppResult_2018 = pd.DataFrame(nppResult_2018.drop(columns='geometry'))
nppResult_2019 = pd.DataFrame(nppResult_2019.drop(columns='geometry'))
nppResult_2020 = pd.DataFrame(nppResult_2020.drop(columns='geometry'))

In [80]:
nppResult_2001.columns

Index(['G04c_001', 'NPP_2001', 'year'], dtype='object')

In [81]:
nppResult_2001.columns = ['G04c_001', 'NPP', 'year']
nppResult_2002.columns = ['G04c_001', 'NPP', 'year']
nppResult_2003.columns = ['G04c_001', 'NPP', 'year']
nppResult_2004.columns = ['G04c_001', 'NPP', 'year']
nppResult_2005.columns = ['G04c_001', 'NPP', 'year']
nppResult_2006.columns = ['G04c_001', 'NPP', 'year']
nppResult_2007.columns = ['G04c_001', 'NPP', 'year']
nppResult_2008.columns = ['G04c_001', 'NPP', 'year']
nppResult_2009.columns = ['G04c_001', 'NPP', 'year']
nppResult_2010.columns = ['G04c_001', 'NPP', 'year']
nppResult_2011.columns = ['G04c_001', 'NPP', 'year']
nppResult_2012.columns = ['G04c_001', 'NPP', 'year']
nppResult_2013.columns = ['G04c_001', 'NPP', 'year']
nppResult_2014.columns = ['G04c_001', 'NPP', 'year']
nppResult_2015.columns = ['G04c_001', 'NPP', 'year']
nppResult_2016.columns = ['G04c_001', 'NPP', 'year']
nppResult_2017.columns = ['G04c_001', 'NPP', 'year']
nppResult_2018.columns = ['G04c_001', 'NPP', 'year']
nppResult_2019.columns = ['G04c_001', 'NPP', 'year']
nppResult_2020.columns = ['G04c_001', 'NPP', 'year']

In [82]:
print(nppResult_2001["NPP"].mean())
print(nppResult_2002["NPP"].mean())
print(nppResult_2003["NPP"].mean())
print(nppResult_2004["NPP"].mean())
print(nppResult_2005["NPP"].mean())
print(nppResult_2006["NPP"].mean())
print(nppResult_2007["NPP"].mean())
print(nppResult_2008["NPP"].mean())
print(nppResult_2009["NPP"].mean())
print(nppResult_2010["NPP"].mean())
print(nppResult_2011["NPP"].mean())
print(nppResult_2012["NPP"].mean())
print(nppResult_2013["NPP"].mean())
print(nppResult_2014["NPP"].mean())
print(nppResult_2015["NPP"].mean())
print(nppResult_2016["NPP"].mean())
print(nppResult_2017["NPP"].mean())
print(nppResult_2018["NPP"].mean())
print(nppResult_2019["NPP"].mean())
print(nppResult_2020["NPP"].mean())

6733.1787
6954.568
6699.3506
6941.993
6611.0015
6508.4023
7125.2705
7066.6777
7038.5234
6502.153
6573.422
6718.171
6813.1533
7084.9976
7307.392
7093.5884
7038.908
7051.997
7239.1533
7185.7896


In [83]:
longNppDF = pd.concat(
    [nppResult_2001, nppResult_2002, nppResult_2003, nppResult_2004, nppResult_2005, nppResult_2006, nppResult_2007, nppResult_2008,
    nppResult_2009, nppResult_2010, nppResult_2011, nppResult_2012, nppResult_2013, nppResult_2014, nppResult_2015, nppResult_2016,
    nppResult_2017, nppResult_2018, nppResult_2019, nppResult_2020]
)

In [84]:
del nppResult_2001
del nppResult_2002
del nppResult_2003
del nppResult_2004
del nppResult_2005
del nppResult_2006
del nppResult_2007
del nppResult_2008
del nppResult_2009
del nppResult_2010
del nppResult_2011
del nppResult_2012
del nppResult_2013
del nppResult_2014
del nppResult_2015
del nppResult_2016
del nppResult_2017
del nppResult_2018
del nppResult_2019
del nppResult_2020

import gc
gc.collect()

150

In [85]:
longNppDF = longNppDF.set_index(['G04c_001','year'])
longNppDF.mean()

NPP    6914.380371
dtype: float32

In [87]:
fileList = glob.glob(aimFolder + "/temp/*.tif")
for filename in fileList:
    os.remove(filename)

os.rmdir(aimFolder + "\\temp")

In [88]:
longNppDF.head()

,,NPP
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


## NTL extraction
Night Time Light

In [3]:
import os
import zipfile
import glob
import rasterio
from osgeo import gdal
import geopandas as gpd
import numpy as np

In [4]:
aimFolder = "F:\\17_Article\\01_Data\\03_NTL_VIIRS"
os.mkdir(aimFolder + "\\temp")

In [12]:
def ntlExtraction(year, aimFolder, coords_extration):
    #This funciton is try to extract ntl
    #filename = aimFolder + "\\" + str(year) + ".zip"
    #with zipfile.ZipFile(filename, "r") as zip_ref:
    #    zip_ref.extractall(aimFolder + "\\temp")
    aimRaster = glob.glob(aimFolder + "\\temp\\*" + str(year) + ".tif")
    aimRaster = aimRaster[0]
    rasterFile = rasterio.open(aimRaster)
    rasterArray = rasterFile.read(1)
    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
    
    valueArray = np.array(valueArray)
    coords_extration['NTL_'+str(year)] = valueArray
    return coords_extration

In [16]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

ntlResult = Parallel(n_jobs=2)(delayed(ntlExtraction)(int(year), aimFolder, coords_extration) for year in np.linspace(2002, 2020, 19))

In [36]:
ntlResult_2001 = ntlExtraction(int(2001), aimFolder, coords_extration)

In [37]:
ntlResult_2002 = ntlResult[0]
ntlResult_2003 = ntlResult[1]
ntlResult_2004 = ntlResult[2]
ntlResult_2005 = ntlResult[3]
ntlResult_2006 = ntlResult[4]
ntlResult_2007 = ntlResult[5]
ntlResult_2008 = ntlResult[6]
ntlResult_2009 = ntlResult[7]
ntlResult_2010 = ntlResult[8]
ntlResult_2011 = ntlResult[9]
ntlResult_2012 = ntlResult[10]
ntlResult_2013 = ntlResult[11]
ntlResult_2014 = ntlResult[12]
ntlResult_2015 = ntlResult[13]
ntlResult_2016 = ntlResult[14]
ntlResult_2017 = ntlResult[15]
ntlResult_2018 = ntlResult[16]
ntlResult_2019 = ntlResult[17]
ntlResult_2020 = ntlResult[18]

In [38]:
ntlResult_2001['year'] = 2001
ntlResult_2002['year'] = 2002
ntlResult_2003['year'] = 2003
ntlResult_2004['year'] = 2004
ntlResult_2005['year'] = 2005
ntlResult_2006['year'] = 2006
ntlResult_2007['year'] = 2007
ntlResult_2008['year'] = 2008
ntlResult_2009['year'] = 2009
ntlResult_2010['year'] = 2010
ntlResult_2011['year'] = 2011
ntlResult_2012['year'] = 2012
ntlResult_2013['year'] = 2013
ntlResult_2014['year'] = 2014
ntlResult_2015['year'] = 2015
ntlResult_2016['year'] = 2016
ntlResult_2017['year'] = 2017
ntlResult_2018['year'] = 2018
ntlResult_2019['year'] = 2019
ntlResult_2020['year'] = 2020

In [40]:
ntlResult_2001 = pd.DataFrame(ntlResult_2001.drop(columns='geometry'))
ntlResult_2002 = pd.DataFrame(ntlResult_2002.drop(columns='geometry'))
ntlResult_2003 = pd.DataFrame(ntlResult_2003.drop(columns='geometry'))
ntlResult_2004 = pd.DataFrame(ntlResult_2004.drop(columns='geometry'))
ntlResult_2005 = pd.DataFrame(ntlResult_2005.drop(columns='geometry'))
ntlResult_2006 = pd.DataFrame(ntlResult_2006.drop(columns='geometry'))
ntlResult_2007 = pd.DataFrame(ntlResult_2007.drop(columns='geometry'))
ntlResult_2008 = pd.DataFrame(ntlResult_2008.drop(columns='geometry'))
ntlResult_2009 = pd.DataFrame(ntlResult_2009.drop(columns='geometry'))
ntlResult_2010 = pd.DataFrame(ntlResult_2010.drop(columns='geometry'))
ntlResult_2011 = pd.DataFrame(ntlResult_2011.drop(columns='geometry'))
ntlResult_2012 = pd.DataFrame(ntlResult_2012.drop(columns='geometry'))
ntlResult_2013 = pd.DataFrame(ntlResult_2013.drop(columns='geometry'))
ntlResult_2014 = pd.DataFrame(ntlResult_2014.drop(columns='geometry'))
ntlResult_2015 = pd.DataFrame(ntlResult_2015.drop(columns='geometry'))
ntlResult_2016 = pd.DataFrame(ntlResult_2016.drop(columns='geometry'))
ntlResult_2017 = pd.DataFrame(ntlResult_2017.drop(columns='geometry'))
ntlResult_2018 = pd.DataFrame(ntlResult_2018.drop(columns='geometry'))
ntlResult_2019 = pd.DataFrame(ntlResult_2019.drop(columns='geometry'))
ntlResult_2020 = pd.DataFrame(ntlResult_2020.drop(columns='geometry'))

In [41]:
ntlResult_2001.columns

Index(['G04c_001', 'NTL_2001', 'year'], dtype='object')

In [45]:
ntlResult_2001.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2002.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2003.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2004.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2005.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2006.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2007.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2008.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2009.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2010.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2011.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2012.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2013.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2014.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2015.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2016.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2017.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2018.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2019.columns = ['G04c_001', 'NTL', 'year']
ntlResult_2020.columns = ['G04c_001', 'NTL', 'year']

In [57]:
ntlResult_2020.NTL[ntlResult_2020["NTL"]<0] = 0
ntlResult_2020["NTL"].mean()
ntlResult_2019.NTL[ntlResult_2019["NTL"]<0] = 0
ntlResult_2019["NTL"].mean()


C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_12728\1752473119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ntlResult_2020.NTL[ntlResult_2020["NTL"]<0] = 0
C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_12728\1752473119.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ntlResult_2019.NTL[ntlResult_2019["NTL"]<0] = 0


1.1114916

In [60]:
print(ntlResult_2001["NTL"].mean())
print(ntlResult_2002["NTL"].mean())
print(ntlResult_2003["NTL"].mean())
print(ntlResult_2004["NTL"].mean())
print(ntlResult_2005["NTL"].mean())
print(ntlResult_2006["NTL"].mean())
print(ntlResult_2007["NTL"].mean())
print(ntlResult_2008["NTL"].mean())
print(ntlResult_2009["NTL"].mean())
print(ntlResult_2010["NTL"].mean())
print(ntlResult_2011["NTL"].mean())
print(ntlResult_2012["NTL"].mean())
print(ntlResult_2013["NTL"].mean())
print(ntlResult_2014["NTL"].mean())
print(ntlResult_2015["NTL"].mean())
print(ntlResult_2016["NTL"].mean())
print(ntlResult_2017["NTL"].mean())
print(ntlResult_2018["NTL"].mean())
print(ntlResult_2019["NTL"].mean())
print(ntlResult_2020["NTL"].mean())

1.6480222
1.6131924
1.4684126
1.2629389
1.5879596
1.575272
1.6215699
1.4386377
1.3462512
1.0992323
1.3615805
1.3009588
1.061045
1.0638536
1.0337858
1.0159932
1.1423855
1.1576278
1.1114916
1.0489889


In [61]:
longNtlDF = pd.concat(
    [ntlResult_2001, ntlResult_2002, ntlResult_2003, ntlResult_2004, ntlResult_2005, ntlResult_2006, ntlResult_2007, ntlResult_2008,
    ntlResult_2009, ntlResult_2010, ntlResult_2011, ntlResult_2012, ntlResult_2013, ntlResult_2014, ntlResult_2015, ntlResult_2016,
    ntlResult_2017, ntlResult_2018, ntlResult_2019, ntlResult_2020]
)

In [64]:
del ntlResult_2001
del ntlResult_2002
del ntlResult_2003
del ntlResult_2004
del ntlResult_2005
del ntlResult_2006
del ntlResult_2007
del ntlResult_2008
del ntlResult_2009
del ntlResult_2010
del ntlResult_2011
del ntlResult_2012
del ntlResult_2013
del ntlResult_2014
del ntlResult_2015
del ntlResult_2016
del ntlResult_2017
del ntlResult_2018
del ntlResult_2019
del ntlResult_2020

import gc
gc.collect()

176

In [62]:
longNtlDF = longNtlDF.set_index(['G04c_001','year'])
longNtlDF.head()

,,NTL
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


In [63]:

longNtlDF.mean()

NTL    1.297961
dtype: float32

In [65]:
del ntlResult
gc.collect()

156

In [71]:
fileList = glob.glob(aimFolder + "\\temp\\*")
#fileList = fileList[1:]
for filename in fileList:
    os.remove(filename)

#os.rmdir(aimFolder + "\\temp")

PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'F:\\17_Article\\01_Data\\03_NTL_VIIRS\\temp\\LongNTL_2001.tif'

In [ ]:
os.rmdir(aimFolder + "\\temp")

In [89]:
longNtlDF.head()

,,NTL
G04c_001,year,
303660001,2001,0.0
303660002,2001,0.0
303660003,2001,0.0
303660004,2001,0.0
303660011,2001,0.0


# Temperature Extraction
Surf_Temp_8Days_1Km (M*D11A2)

In [96]:
from IPython import get_ipython
get_ipython().magic('reset -sf')

In [97]:
import os
import glob
from osgeo import gdal
import numpy as np
import gc
import geopandas as gpd
import rasterio
import math

In [99]:
aimFolder = "F:\\17_Article\\01_Data\\08_Temperature\\Surf_Temp_8Days_1Km_v6"
os.mkdir(aimFolder + "\\temp")
#year = 2015

In [107]:
def dayTimeTemperatureRaster(year, aimFolder):
### Day time Temp
    dayTimeTemperatureFileList = glob.glob(aimFolder + "\\LST_Day_1km\\M*" + str(year) + "*.tif")

    dayTimeTemperatureRasterList = []
    for dayTimeTemperatureFile in dayTimeTemperatureFileList:
        dayTimeTemperatureRasterRaw = gdal.Open(dayTimeTemperatureFile, gdal.GA_ReadOnly)
        dayTimeTemperatureRaster = dayTimeTemperatureRasterRaw.ReadAsArray()
        dayTimeTemperatureRasterRaw = None
        dayTimeTemperatureRaster = dayTimeTemperatureRaster.astype("float")
        dayTimeTemperatureRaster[dayTimeTemperatureRaster == 0] = np.nan
        dayTimeTemperatureRaster = dayTimeTemperatureRaster * 0.02 - 273.16
        dayTimeTemperatureRasterList.append(dayTimeTemperatureRaster)

    meanDayTimeTemperatureRaster = np.nanmean(dayTimeTemperatureRasterList, axis = 0)
    stdDayTimeTemperatureRaster = np.nanstd(dayTimeTemperatureRasterList, axis = 0)
    dayTimeTemperatureRasterList = None
    gc.collect()

    src_dataset = gdal.Open(dayTimeTemperatureFileList[0], gdal.GA_ReadOnly)
    geotransform = src_dataset.GetGeoTransform()
    spatialreference = src_dataset.GetProjection()
    ncol = meanDayTimeTemperatureRaster.shape[1]
    nrow = meanDayTimeTemperatureRaster.shape[0]
    nband = 1
    src_dataset = None

    # write the tif files
    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(aimFolder + "\\temp\\" + str(year) + "_daytime_mean.tif", ncol, nrow, nband, gdal.GDT_Float32) 
    #### ^^^^^ Change this line
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(meanDayTimeTemperatureRaster)
    dst_dataset = None

    # write the tif files
    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(aimFolder + "\\temp\\" + str(year) + "_daytime_std.tif", ncol, nrow, nband, gdal.GDT_Float32) 
    #### ^^^^^ Change this line
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(stdDayTimeTemperatureRaster)
    dst_dataset = None

    ### resample and reproject
    raster_rprj = gdal.Warp(aimFolder + "\\temp\\" + str(year) + "_daytime_mean_re.tif", 
                            aimFolder + "\\temp\\" + str(year) + "_daytime_mean.tif", dstSRS = "EPSG:4326",
                            xRes = 0.008, yRes = 0.008, resampleAlg = "average", srcNodata = math.nan)
    raster_rprj = None

    raster_rprj = gdal.Warp(aimFolder + "\\temp\\" + str(year) + "_daytime_std_re.tif", 
                            aimFolder + "\\temp\\" + str(year) + "_daytime_std.tif", dstSRS = "EPSG:4326",
                            xRes = 0.008, yRes = 0.008, resampleAlg = "average", srcNodata = math.nan)
    raster_rprj = None


In [113]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

#Parallel(n_jobs=2)(delayed(dayTimeTemperatureRaster)(int(year), aimFolder) for year in np.linspace(2001, 2020, 20))
#parallel unavailable

for year in np.linspace(2003, 2020, 18):
    dayTimeTemperatureRaster(int(year), aimFolder)

C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_12728\1949070710.py:15: RuntimeWarning: Mean of empty slice
  meanDayTimeTemperatureRaster = np.nanmean(dayTimeTemperatureRasterList, axis = 0)
d:\Anaconda3\envs\Usegdal\lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [115]:
gc.collect()

154

In [118]:
def nightTimeTemperatureRaster(year, aimFolder):
    ### Night time Temp
    nightTimeTemperatureFileList = glob.glob(aimFolder + "\\LST_Night_1km\\M*" + str(year) + "*.tif")

    nightTimeTemperatureRasterList = []
    for nightTimeTemperatureFile in nightTimeTemperatureFileList:
        nightTimeTemperatureRasterRaw = gdal.Open(nightTimeTemperatureFile, gdal.GA_ReadOnly)
        nightTimeTemperatureRaster = nightTimeTemperatureRasterRaw.ReadAsArray()
        nightTimeTemperatureRasterRaw = None
        nightTimeTemperatureRaster = nightTimeTemperatureRaster.astype("float")
        nightTimeTemperatureRaster[nightTimeTemperatureRaster == 0] = np.nan
        nightTimeTemperatureRaster = nightTimeTemperatureRaster * 0.02 - 273.16
        nightTimeTemperatureRasterList.append(nightTimeTemperatureRaster)

    meanNightTimeTemperatureRaster = np.nanmean(nightTimeTemperatureRasterList, axis = 0)
    stdNightTimeTemperatureRaster = np.nanstd(nightTimeTemperatureRasterList, axis = 0)
    nightTimeTemperatureRasterList = None
    gc.collect()

    src_dataset = gdal.Open(nightTimeTemperatureFileList[0], gdal.GA_ReadOnly)
    geotransform = src_dataset.GetGeoTransform()
    spatialreference = src_dataset.GetProjection()
    ncol = meanNightTimeTemperatureRaster.shape[1]
    nrow = meanNightTimeTemperatureRaster.shape[0]
    nband = 1
    src_dataset = None

    # write the tif files
    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(aimFolder + "\\temp\\" + str(year) + "_nighttime_mean.tif", ncol, nrow, nband, gdal.GDT_Float32) 
    #### ^^^^^ Change this line
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(meanNightTimeTemperatureRaster)
    dst_dataset = None

    # write the tif files
    driver = gdal.GetDriverByName("GTiff")
    dst_dataset = driver.Create(aimFolder + "\\temp\\" + str(year) + "_nighttime_std.tif", ncol, nrow, nband, gdal.GDT_Float32) 
    #### ^^^^^ Change this line
    dst_dataset.SetGeoTransform(geotransform)
    dst_dataset.SetProjection(spatialreference)
    dst_dataset.GetRasterBand(1).WriteArray(stdNightTimeTemperatureRaster)
    dst_dataset = None

    ### resample and reproject
    raster_rprj = gdal.Warp(aimFolder + "\\temp\\" + str(year) + "_nighttime_mean_re.tif", 
                            aimFolder + "\\temp\\" + str(year) + "_nighttime_mean.tif", dstSRS = "EPSG:4326",
                            xRes = 0.008, yRes = 0.008, resampleAlg = "average", srcNodata = math.nan)
    raster_rprj = None

    raster_rprj = gdal.Warp(aimFolder + "\\temp\\" + str(year) + "_nighttime_std_re.tif", 
                            aimFolder + "\\temp\\" + str(year) + "_nighttime_std.tif", dstSRS = "EPSG:4326",
                            xRes = 0.008, yRes = 0.008, resampleAlg = "average", srcNodata = math.nan)
    raster_rprj = None

In [119]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

for year in np.linspace(2001, 2020, 20):
    nightTimeTemperatureRaster(int(year), aimFolder)

C:\Users\li.chao.987@s.kyushu-u.ac.jp\AppData\Local\Temp\ipykernel_12728\342181939.py:15: RuntimeWarning: Mean of empty slice
  meanNightTimeTemperatureRaster = np.nanmean(nightTimeTemperatureRasterList, axis = 0)


In [120]:
coords_extration = gpd.read_file("F:/17_Article/01_Data/00_mesh/Mesh500/mergedPointMesh500m.shp")

In [121]:
def coordExtractionFromRaster(RasterName, GeoPandasDataFrame, NewColumnName):
    rasterFile = rasterio.open(RasterName)
    coords_extration = GeoPandasDataFrame.copy()
    rasterArray = rasterFile.read(1)
    
    valueArray = []
    for point in coords_extration['geometry']:
        x = point.xy[0][0]
        y = point.xy[1][0]
        row, col = rasterFile.index(x, y)
        valueArray.append(rasterArray[row, col])
        
    valueArray = np.array(valueArray)
    coords_extration[NewColumnName] = valueArray
    rasterFile = None
    return coords_extration

In [124]:

from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import glob

dayTimeTempMeanResult = Parallel(n_jobs=8)(delayed(coordExtractionFromRaster)(aimFolder + "\\temp\\" + str(int(year)) + "_daytime_mean_re.tif", coords_extration, 'dayTimeMeanTemp') for year in np.linspace(2001, 2020, 20))

In [126]:
dayTimeTempMeanResult[0].columns

Index(['G04c_001', 'geometry', 'dayTimeMeanTemp'], dtype='object')

In [131]:
def mergeParallelDf(DF):
    singleDF_0 = DF[0]
    singleDF_1 = DF[1]
    singleDF_2 = DF[2]
    singleDF_3 = DF[3]
    singleDF_4 = DF[4]
    singleDF_5 = DF[5]
    singleDF_6 = DF[6]
    singleDF_7 = DF[7]
    singleDF_8 = DF[8]
    singleDF_9 = DF[9]
    singleDF_10 = DF[10]
    singleDF_11 = DF[11]
    singleDF_12 = DF[12]
    singleDF_13 = DF[13]
    singleDF_14 = DF[14]
    singleDF_15 = DF[15]
    singleDF_16 = DF[16]
    singleDF_17 = DF[17]
    singleDF_18 = DF[18]
    singleDF_19 = DF[19]

    singleDF_0['year'] = 2001
    singleDF_1['year'] = 2002 
    singleDF_2['year'] = 2003 
    singleDF_3['year'] = 2004 
    singleDF_4['year'] = 2005 
    singleDF_5['year'] = 2006 
    singleDF_6['year'] = 2007 
    singleDF_7['year'] = 2008 
    singleDF_8['year'] = 2009 
    singleDF_9['year'] = 2010 
    singleDF_10['year'] = 2011
    singleDF_11['year'] = 2012 
    singleDF_12['year'] = 2013 
    singleDF_13['year'] = 2014 
    singleDF_14['year'] = 2015 
    singleDF_15['year'] = 2016 
    singleDF_16['year'] = 2017 
    singleDF_17['year'] = 2018 
    singleDF_18['year'] = 2019 
    singleDF_19['year'] = 2020

    mergeDF = pd.concat([
        singleDF_0, singleDF_1, singleDF_2, singleDF_3, singleDF_4, singleDF_5, singleDF_6, singleDF_7, singleDF_8, singleDF_9,
        singleDF_10, singleDF_11, singleDF_12, singleDF_13, singleDF_14, singleDF_15, singleDF_16, singleDF_17, singleDF_18, singleDF_19
    ])

    return mergeDF


In [132]:
import pandas as pd
mergeDayTimeTempMeanResult = mergeParallelDf(dayTimeTempMeanResult)

In [133]:
mergeDayTimeTempMeanResult.columns

Index(['G04c_001', 'geometry', 'dayTimeMeanTemp', 'year'], dtype='object')

In [134]:
mergeDayTimeTempMeanResult = pd.DataFrame(mergeDayTimeTempMeanResult.drop(columns='geometry'))
mergeDayTimeTempMeanResult = mergeDayTimeTempMeanResult[['G04c_001', 'dayTimeMeanTemp', 'year']]
mergeDayTimeTempMeanResult = mergeDayTimeTempMeanResult.set_index(['G04c_001', 'year'])


In [135]:
del dayTimeTempMeanResult
gc.collect()

1278

In [136]:
dayTimeTempStdResult = Parallel(n_jobs=8)(delayed(coordExtractionFromRaster)(aimFolder + "\\temp\\" + str(int(year)) + "_daytime_std_re.tif", coords_extration, 'dayTimeStdTemp') for year in np.linspace(2001, 2020, 20))

In [137]:
mergeDayTimeTempStdResult = mergeParallelDf(dayTimeTempStdResult)
mergeDayTimeTempStdResult = pd.DataFrame(mergeDayTimeTempStdResult.drop(columns='geometry'))
mergeDayTimeTempStdResult = mergeDayTimeTempStdResult[['G04c_001', 'dayTimeStdTemp', 'year']]
mergeDayTimeTempStdResult = mergeDayTimeTempStdResult.set_index(['G04c_001', 'year'])

In [139]:
mergeDayTimeTempStdResult.mean()

dayTimeStdTemp    9.443231
dtype: float32

In [145]:
mergeDayTimeTempMeanResult['dayTimeMeanTemp'].isna().count() - mergeDayTimeTempMeanResult['dayTimeMeanTemp'].count() 

523921

In [148]:
dayTimeTempStdResult[0]['dayTimeStdTemp'].isna().count() - dayTimeTempStdResult[0]['dayTimeStdTemp'].count() 

26241

In [149]:
dayTimeTempStdResult[0]['dayTimeStdTemp'].isna().count() 

1553024

In [152]:
rasterFile = rasterio.open(r"F:\17_Article\01_Data\08_Temperature\Surf_Temp_8Days_1Km_v6\temp\2001_daytime_mean.tif")
rasterFile = rasterFile[::-1,:]
rasterArray = rasterFile.read(1)
x, y = 139.878, 35.860
row, col = rasterFile.index(x, y)
print(row, col)
#print(rasterArray[row, col])


TypeError: 'DatasetReader' object is not subscriptable

In [163]:
coords_extration.head()

,G04c_001,geometry
0,303660001,POINT (136.00313 20.50208)
1,303660002,POINT (136.00937 20.50208)
2,303660003,POINT (136.00313 20.50625)
3,303660004,POINT (136.00937 20.50625)
4,303660011,POINT (136.01562 20.50208)


# Merge and Save Out
now includes longNtlResult and longNppResult

In [92]:
longFinalDf = pd.concat([longNtlDF, longNppDF])

In [93]:
longFinalDf.to_pickle("F:/17_Article/01_Data/98_20yearPickles/02_NtlNppPrecTemp.pkl")

In [153]:
longFinalDf = pd.read_pickle("F:/17_Article/01_Data/98_20yearPickles/02_NtlNppPrecTemp.pkl") 

In [169]:
longFinalDf[pd.isna(longFinalDf.NPP)].count()

NTL    31060480
NPP           0
dtype: int64

In [168]:
longFinalDf[pd.isna(longFinalDf.NTL)].count()

NTL           0
NPP    31060160
dtype: int64

In [170]:
longFinalDf.shape

(62120960, 2)

In [158]:
longFinalDf.head()

,,NTL,NPP
G04c_001,year,,
303660001,2001,0.0,NaN
303660002,2001,0.0,NaN
303660003,2001,0.0,NaN
303660004,2001,0.0,NaN
303660011,2001,0.0,NaN
